# ДЗ1. Адаптация LLM под задачу

## Введение

Дата выдачи: 08.02.2026

Мягкий дедлайн: 23:59MSK 23.02.2026

Жесткий дедлайн: 23:59MSK 28.02.2026

В этом домашнем задании вам предстоит адаптировать одну и ту же LLM под задачу следования инструкциям и сравнить несколько практических подходов:

* **Prompting (baseline):** без обучения, только подбор формата запроса.
* **LoRA fine-tuning:** параметро-эффективное дообучение (можно в 4-bit режиме).
* **Full fine-tuning:** дообучение целиком (обучаем все веса).
* **DPO:** дообучение по предпочтениям на парных данных (chosen/rejected).

В ходе работы вы пройдёте весь *post-training* пайплайн:

1. Подготовка данных и шаблона диалога (chat template)
2. Prompting baseline и набор фиксированных тест-промптов
3. Реализация LoRA-слоя + тесты
4. Встраивание LoRA в модель + sanity-checks
5. SFT-дообучение: LoRA vs full fine-tuning
6. Сравнение качества/стоимости и анализ ошибок
7. DPO по предпочтениям (chosen/rejected)

> В ноутбуке используется `tokenizer.apply_chat_template` для формирования промптов под instruct-модель.
>
> В TRL у тренеров аргумент `tokenizer` был переименован в processing_class (поэтому в работе используется именно он).

**О задании**

Вы берете одну модель и ведете ее через этапы адаптации: prompting -> LoRA-SFT -> full-SFT -> DPO, после чего делаете сравнение на фиксированных промптах и на отложенной выборке (loss). На каждом этапе вы сохраняете результаты (ответы на одинаковых промптах и метрики), чтобы сравнение было честным и воспроизводимым.

**Оценивание и условия**

Максимальный балл за работу - **10**.

Вот несколько правил, который помогут нам сделать работу приятнее и продуктивнее:

* Можно использовать любые свободные источники с обязательным указанием ссылки на них. Если в работе вы используете генеративные модели, их указание обязательно. Также следите за оригинальностью: генеративного кода должно быть не более 60% работы. На усмотрение проверяющего баллы за работу в случае использования сгенерированного кода могут быть снижены.

* Плагиат не допускается. При обнаружении случаев списывания, 0 за работу выставляется всем участникам нарушения, даже если можно установить, кто у кого списал.

* Старайтесь сделать код как можно более оптимальным. В частности, будет штрафоваться использование циклов в тех случаях, когда операцию можно совершить при помощи инструментов библиотек, о которых рассказывалось в курсе.

**Формат сдачи**

Задания сдаются через систему **LMS**. Посылка должна содержать:

- ноутбук homework-01-Username.ipynb
где Username - ваша фамилия латиницей, без пробелов (например, homework-01-Ivanov.ipynb).



## Setting

In [ ]:
# IMPORTANT:
# 1) СНАЧАЛА запустите ноутбук в "CPU-режиме" и проверьте,
#    что базовая часть работает: импорты, загрузка датасета,
#    токенизация, форматирование промпта, 1 короткая генерация.
#
# 2) ТОЛЬКО ПОСЛЕ этого подключайте GPU и грузите модель на CUDA.
#
# Почему так:
# - В Colab подключение GPU начинает "съедать" лимит времени/ресурсов,
#   даже когда вы просто думаете и ничего не считаете.
# - Если сначала отладить код на CPU, вы экономите GPU-время и нервы.
#
# Как подключить GPU:
# Runtime -> Change runtime type -> Hardware accelerator -> GPU

In [1]:
%pip -q install "transformers>=4.43" "datasets>=2.18" "accelerate>=0.33" "trl>=0.9" "bitsandbytes>=0.45"

import os, random, math, time
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    DataCollatorForLanguageModeling
)

SEED = 42
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

device = "cuda" if torch.cuda.is_available() else "cpu"

MODEL_NAME = "Qwen/Qwen3-0.6B"

DTYPE = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
print("device:", device, "| dtype:", DTYPE)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.9/530.9 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 45.1 MB/s eta 0:00:00
device: cuda | dtype: torch.bfloat16


## Задание 0. Данные и вспомогательные функции (1 балл)

**Данные:** `tatsu-lab/alpaca`

В этом ДЗ мы используем датасет `tatsu-lab/alpaca`— это версия Stanford Alpaca, одного из самых известных учебных наборов для **instruction fine-tuning (SFT)**.

Он содержит пары *инструкция -> ответ* (и иногда дополнительный input) в полях `instruction`, `input`, `output`.

Важно понимать, что это **синтетический** датасет: инструкции и ответы были сгенерированы с помощью модели OpenAI `text-davinci-003`, поэтому данные могут быть неидеальными и местами слишком гладкими/шаблонными - это нормальная часть эксперимента.

**Почему мы работаем именно с ним?**

1. **Простой и понятный формат.** Удобно собрать chat-template и быстро запустить SFT без долгой возни с парсингом диалогов.
2. **Небольшой и быстрый для обучения.** Можно брать небольшой срез (несколько тысяч примеров) и уложиться в разумное время даже на ограниченной GPU.
3. **Хорош для честного сравнения режимов post-training.** На одном и том же наборе легко сравнить: prompting vs LoRA-SFT vs full-SFT - и увидеть trade-off качество/стоимость.
4. **Учебная ценность.** Датасет достаточно разнообразный по типам инструкций (объяснения, списки, преобразования, простые задачи), поэтому эффект от fine-tuning обычно заметен даже при коротком обучении.

### Ознакомиться с данными (0.5 балла)

Перед тем как обучать модель, нужно минимально понять, что именно лежит в датасете и как это будет превращаться в диалог.

**Что нужно сделать:**

1. Загрузить `tatsu-lab/alpaca` и вывести:

   - список колонок,
   - размер датасета,
   - 3 случайных примера (с выводом `instruction`, `input`, `output`).

2. Посчитать и вывести (в одном месте, коротко):

   - долю примеров, где `input` пустой,
   - распределение длины `output` (например, min/median/mean/max по числу символов или токенов - на ваш выбор),
   - топ-5 самых частых первых слов в `instruction`.

3. Вручную посмотреть 5 примеров и ответить текстом:

   - какие типы инструкций встречаются чаще всего?
   - есть ли странные или явно низкокачественные ответы? (2–3 предложения)

In [ ]:
# !в сплите можно управлять размером данных
ds = load_dataset("tatsu-lab/alpaca", split="train")

# your code here ┌(ಠ_ಠ)┘

In [ ]:
# and maybe also ٩(⁎❛ᴗ❛⁎)۶

Какие типы инструкций встречаются чаще всего?
Есть ли низкокачественные примеры? (2–3 предложения)

`# your text here (ಠ.ಠ)`

### Вспомогательные функции и базовая загрузка модели (0.5 балла)


Дальше в ДЗ вам будет постоянно нужно:

1. **собирать чат-промпт** из сообщений (user / assistant / system),
2. **быстро генерировать ответы** базовой/дообученной модели,
3. **превращать Alpaca-пример** в готовую строку `text` для SFT (`train_ds_txt/test_ds_txt`).

Чтобы не копировать один и тот же код по всему ноутбуку, мы предлагаем сделать несколько вспомогательных функций.

> **Это не строго обязательно.** Если вам проще - вы можете не писать эти функции и делать всё в лоб прямо в следующих заданиях. Но с функциями будет проще поддерживать код и сравнивать результаты.
>
> Если вы примете решение их не реализовывать, напишите комментарий об этом для проверяющего.

**Шаг 1. Загрузите токенайзер и базовую модель** `Qwen/Qwen3-0.6B` с [HuggingFace](https://huggingface.co/Qwen/Qwen3-0.6B).
Проверьте, что модель переводится в `eval()` и умеет генерировать (в следующем шаге).

---



In [7]:
# your code here (⌐■_■)

# Возможно, потребуется явно выставить pad_token
# print("Tokenizer pad_token:", tokenizer.pad_token, "| eos_token:", tokenizer.eos_token)

**Шаг 2. Подготовьте функции:**

- `make_chat(messages, add_generation_prompt=True)` - собирает строку промпта из списка сообщений.
- `make_user_assistant_pair(...)` - удобный конструктор списка сообщений.
- `generate_text(...)` - генерирует ответ модели на один запрос.
- `alpaca_to_text(ex)` - превращает одну запись Alpaca в поле `{"text": ...}` для обучения.

In [ ]:
# Chat-template

def make_chat(messages, add_generation_prompt: bool = True) -> str:
    """
    Универсальная сборка промпта:
    - если у токенайзера есть chat_template -> используем apply_chat_template
    - иначе -> делаем простой fallback-формат

    messages: list[dict], пример:
      [{"role":"system","content":"..."}, {"role":"user","content":"..."}]
    """

    # your code here (づ｡◕‿‿◕｡)づ

    pass

In [ ]:
# Удобный конструктор сообщений

def make_user_assistant_pair(
    user_text: str,
    assistant_text: str | None,
    system_text: str | None = None
):
    """
    Собирает список сообщений:
    (опционально) system -> user -> (опционально) assistant
    """

    # your code here ٩(⁎❛ᴗ❛⁎)۶

    pass

In [ ]:
# Генерация текста (один промпт в один ответ)

@torch.no_grad()
def generate_text(
    model,
    user_text: str,
    system_text: str | None = None,
    max_new_tokens=128,
    temperature=0.7,
    top_p=0.9
) -> str:

    """
    Генерация ответа модели на один запрос.
    Возвращает decoded строку (без спец-токенов).
    """

    # your code here (ง •̀_•́)ง

    pass

In [ ]:
def alpaca_to_text(ex):
    """
    Делает из Alpaca-примера поле text (готовое для SFTTrainer),
    используя make_chat(..., add_generation_prompt=False).

    Alpaca поля:
      ex["instruction"], ex.get("input",""), ex["output"]
    """

    # your code here ┌(ಠ_ಠ)┘

    pass

**Шаг 3. Мини-проверки (sanity-check):**

- напечатайте пример результата `alpaca_to_text` (первые 400–600 символов),
- сгенерируйте ответ базовой модели на простой промпт (1-2 примера),
- убедитесь, что ничего не падает.

In [ ]:
# Проверка: генерация на базовой модели
print(generate_text(base_model, "Explain what overfitting is in 2 bullet points."))

In [ ]:
# Проверка: что alpaca_to_text возвращает строку
ex0 = ds[0]
tmp = alpaca_to_text(ex0)
print(type(tmp), list(tmp.keys()))
print(tmp["text"][:600])

## Задание 1. Prompting baseline (0.5 балла)

Прежде чем обучать модель (LoRA / full SFT), нужно получить честный нулевой уровень (*baseline*): как ведёт себя та же самая модель, если мы вообще не трогаем веса и меняем только формат запроса.

Это важно по двум причинам:

1. **Контрольная точка для сравнения.** Если после SFT ответы стали лучше, мы должны понимать: это действительно эффект обучения, или мы могли бы получить почти то же самое простым промптингом.

2. **Проверка стабильности и воспроизводимости.** Фиксированный набор тест-промптов станет вашим мини-бенчмарком, на котором вы потом сравните baseline vs LoRA-SFT vs full-SFT.


Теперь давайте:

1. Соберем фиксированный набор тест-промптов `EVAL_PROMPTS` (минимум 15).
   Промпты должны быть **разнотипными**: объяснение, список, преобразование текста, простой код, вопрос на здравый смысл и т.п.

2. Для каждого промпта получим ответы базовой модели в двух режимах:

* **без system** (обычный user - assistant)
* **с system** (добавляем системную инструкцию, например "кратко, структурно")

3. Сформируем результаты в таблицу `baseline_df` со столбцами:

* `prompt`
* `no_system`
* `with_system`

> Предполагается, что из **Задания 0** у вас уже есть:
>*Курсив*
> `base_model`, `generate_text()` и рабочий токенайзер/модель.


In [ ]:
# Prompting baseline

EVAL_PROMPTS = [
    "Explain overfitting in 3 bullet points.",
    "Write a Python function that checks if a number is prime.",
    "Give a short summary of the idea of gradient descent.",

    # your code here: добавьте минимум 12 своих промптов (✿◠‿◠)

]

SYSTEM_INSTR = "Отвечай кратко, структурно, без воды. Если уместно — списком."

baseline_rows = []

for p in EVAL_PROMPTS:

    # your code here (ง •̀_•́)ง

    pass

baseline_df = pd.DataFrame(baseline_rows)

In [ ]:
# your code here: покажите первые строки baseline_df

Напишите коротко текстом:

* В каких типах промптов `SYSTEM_INSTR` реально помогает?
* Где почти не влияет?

`# your text here (ಠ.ಠ)`

## Задание 2. Реализация LoRA слоя + тесты (1.5 балла)

В следующих заданиях вы будете дообучать модель с LoRA. Чтобы понимать, что именно меняется в модели и почему LoRA экономит ресурсы, мы сначала реализуем LoRA **вручную** для одного базового кирпичика - `nn.Linear`.

Это дает две вещи:

1. **Понимание механики:** LoRA - это не магия из библиотеки, а конкретная добавка низкорангового обновления к уже обученному слою.
2. **Контроль корректности:** тесты гарантируют, что слой ведет себя правильно (особенно важное свойство `B=0 -> слой = base`).

**Задание:**

Реализуйте LoRA-обертку для `nn.Linear` по формуле:

$$
y = Wx + \frac{\alpha}{r}\cdot (xAB)
$$

Где:

- `W` - **замороженные** веса исходного слоя
- `A` и `B` - **обучаемые** матрицы низкого ранга (`r`)
- при `B = 0` слой должен в точности совпадать с исходным линейным слоем
- градиенты должны идти только в `A` и `B` (веса `W` не обучаются)

In [ ]:
# LoRA layer

class LoRALinear(nn.Module):
    def __init__(self, base: nn.Linear, r: int = 8, alpha: float = 16.0, dropout: float = 0.0):
        super().__init__()
        assert isinstance(base, nn.Linear)

        self.base = base
        self.r = r
        self.alpha = alpha
        self.scaling = alpha / r
        self.drop = nn.Dropout(dropout)

        # 1) Freeze base weights
        # 2) Create LoRA params A and B
        # 3) Init A (B stays zeros to satisfy B=0 => base)

        # your code here (ง •̀_•́)ง

        pass

    def forward(self, x):

        # your code here ٩(⁎❛ᴗ❛⁎)۶

        pass

In [ ]:
# Тесты (должны проходить)

def test_lora_basic():
    torch.manual_seed(0)

    lin = nn.Linear(4, 3, bias=False)
    lin.weight.data.fill_(1.0)

    lora = LoRALinear(lin, r=2, alpha=2.0, dropout=0.0)

    x = torch.randn(5, 4)
    y_base = lin(x)
    y_lora = lora(x)

    # Property: B=0 => LoRA equals base layer
    assert torch.allclose(y_base, y_lora, atol=1e-6), "B=0 => LoRA must equal base layer"

    # Gradients should flow into A and B
    loss = y_lora.sum()
    loss.backward()
    assert lora.A.grad is not None and lora.B.grad is not None, "No grads for LoRA params"

test_lora_basic()
print("LoRA tests: OK")

**Мини-вопрос**

Почему мы инициализируем `B` нулями, а `A` - случайно? Что это даёт в начале обучения?

`# your text here (ಠ.ಠ)`


## Задание 3. Встраивание LoRA в модель + sanity-check (1 балл)

Вы уже реализовали LoRA для одного слоя. Теперь нужно сделать следующий инженерный шаг:

1. **встроить LoRA внутрь реальной LLM**, заменив нужные `nn.Linear` слои в attention-блоке,
2. убедиться, что мы действительно обучаем только LoRA-параметры, а не случайно разморозили половину модели.

Это важный sanity-check перед любым обучением: иначе вы можете потратить часы GPU и не понять, *что именно* вы обучали.

**Задание:**

1. Загрузите модель `MODEL_NAME`, если еще нет.
2. Встройте LoRA минимум в слои attention-проекций:

   - обязательно: `q_proj`, `k_proj`, `v_proj`
   - опционально (приветствуется): `o_proj`
3. Посчитайте:

   - число обучаемых параметров
   - долю обучаемых параметров от общего числа
4. Сделайте sanity-check:

   - среди всех параметров с `requires_grad=True` должны быть только LoRA-матрицы `A` и `B`.

> Предполагается, что из Задания 0 у вас уже есть: `MODEL_NAME`, `DTYPE`, а из Задания 2 - класс `LoRALinear`.

In [ ]:
# Inject LoRA into model

from transformers import AutoModelForCausalLM

def set_submodule(root: nn.Module, path: str, new_module: nn.Module):

    """
    Заменяет подмодуль по строковому пути вида "model.layers.0.self_attn.q_proj".
    """

    parts = path.split(".")
    parent = root
    for p in parts[:-1]:
        parent = getattr(parent, p)
    setattr(parent, parts[-1], new_module)

def inject_lora(
    model: nn.Module,
    target_suffixes=("q_proj","k_proj","v_proj","o_proj"),
    r=8,
    alpha=16,
    dropout=0.05
):

    # your code here (♡ (´｡• ω •｡`))

    return replaced

In [ ]:
# Load fresh model for LoRA experiment
lora_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=DTYPE
)

# Inject LoRA
num_replaced = inject_lora(lora_model, r=8, alpha=16, dropout=0.05)
print("Replaced линейных слоёв:", num_replaced)

In [ ]:
# Count trainable params

trainable = sum(p.numel() for p in lora_model.parameters() if p.requires_grad)
total = sum(p.numel() for p in lora_model.parameters())
print("Trainable params:", trainable)
print("Total params:", total)
print("Trainable share:", trainable / total)

In [ ]:
# Sanity-check: only A/B are trainable

bad = []
for n, p in lora_model.named_parameters():
    if p.requires_grad:

        # your code here (⊙_⊙)
        # Логика: если имя не заканчивается на ".A" или ".B" -> добавить в bad

        pass

print("Non-LoRA trainable params:", bad[:20], "count=", len(bad))
assert len(bad) == 0, "Нашлись обучаемые параметры не из LoRA!"
print("sanity-check: OK")

**Вопрос:**

Почему LoRA обычно ставят именно в `q_proj/k_proj/v_proj` (и иногда `o_proj`)? Что это дает по сравнению с "вставить LoRA куда угодно"?

`# your text here (ಠ.ಠ)`

## Задание 4. SFT обучение с LoRA (2 балла)

Дообучите LoRA-модель на `train_ds_txt` (поле `text`) и покажите, что:

- training loss убывает
- на фиксированном наборе `EVAL_PROMPTS` ответы меняются осмысленно по сравнению с baseline

Можно использовать `trl.SFTTrainer` (рекомендуется) или свой тренировочный цикл.

In [ ]:
from trl import SFTTrainer, SFTConfig

lora_model.config.use_cache = False
lora_model.train()

cfg_lora = SFTConfig(
    output_dir="out_sft_lora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    max_steps=300, # можно меньше/больше
    logging_steps=25,
    save_strategy="no",
    fp16=(DTYPE == torch.float16)
)

trainer_lora = SFTTrainer(
    model=lora_model,
    args=cfg_lora,
    train_dataset=train_ds_txt,
    eval_dataset=test_ds_txt,
    tokenizer=tokenizer,
    dataset_text_field="text",
)

# your code here: trainer_lora.train() ᕙ(⇀‸↼‶)ᕗ

In [ ]:
# your code here:

# 1) Сгенерируйте ответы LoRA-SFT на EVAL_PROMPTS
# 2) Сохраните табличку prompt / baseline_with_system / lora_sft

# ヽ(♡‿♡)ノ

lora_rows = []
for p in EVAL_PROMPTS:
    base_out = baseline_df[baseline_df["prompt"] == p]["with_system"].values[0]
    lora_out = generate_text(lora_model, p, system_text=SYSTEM_INSTR)
    lora_rows.append({"prompt": p, "baseline_with_system": base_out, "lora_sft": lora_out})

cmp_df = pd.DataFrame(lora_rows)
cmp_df.head()

In [ ]:
# your code here: сохранить промежуточную таблицу
# (⌐■_■)

cmp_df.to_csv("compare_baseline_lora.csv", index=False)
print("saved: compare_baseline_lora.csv")

## Задание 5. Full fine-tuning (2 балла)

На baseline-промптинге мы увидели, что можно выжать из модели без обучения. Теперь задача - сделать **instruction fine-tuning (SFT)** и проверить, что:

1. модель действительно обучается (loss убывает),
2. поведение меняется осмысленно на том же наборе `EVAL_PROMPTS` (то есть улучшение - не кажется, а видно при прямом сравнении).

Это ключевой этап: дальше вы будете сравнивать LoRA-SFT с full-SFT, поэтому нужно аккуратно зафиксировать результат LoRA.

**Этапы:**
1. Соберите конфиг обучения SFTConfig.
2. Запустите SFT-обучение LoRA-модели на `train_ds_txt` (поле `text`).
   Можно использовать `trl.SFTTrainer` (рекомендуется) или свой цикл.
3. Покажите, что training loss убывает (достаточно лога `logging_steps` и/или графика).
4. Сравните ответы baseline vs LoRA-SFT на `EVAL_PROMPTS`:

   * используйте тот же `SYSTEM_INSTR`, что в baseline,
   * соберите таблицу `cmp_df` со столбцами:

     * `prompt`
     * `baseline_with_system`
     * `lora_sft`

5. Покажите `cmp_df.head()` и 3–5 примеров до/после текстом или таблицей.



> Предполагается, что из предыдущих заданий у вас уже есть:
> `lora_model`, `tokenizer`, `train_ds_txt`, `test_ds_txt`, `EVAL_PROMPTS`, `baseline_df`, `SYSTEM_INSTR`, `generate_text()`.

**Ограничения на конфиг**

Вы можете выбирать любые значения, но соблюдайте рамки:

* `max_steps`: [100, 400]
* `learning_rate`: в диапазоне [5e-5, 5e-4]
* `per_device_train_batch_size`: 1 или 2
* `gradient_accumulation_steps`: ≥ 4
* `fp16`: включайте, если `DTYPE == torch.float16` (иначе выключайте)
* `save_strategy="no"` (чтобы не плодить артефакты)

**Вопрос на понимание**

Перед запуском обучения ответьте:

* почему вы выбрали именно такой `learning_rate` и `max_steps`?
* что будет, если сделать `learning_rate` слишком большим?

`# your text here (ಠ.ಠ)`

In [ ]:
# LoRA-SFT
from trl import SFTTrainer, SFTConfig

# Prepare model for training
lora_model.config.use_cache = False
lora_model.train()

# Соберите cfg_lora сами

# your code here (ง •̀_•́)ง

# Подсказка: параметры, которые точно нужны:
# - output_dir
# - per_device_train_batch_size
# - gradient_accumulation_steps
# - learning_rate
# - max_steps
# - logging_steps
# - save_strategy
# - fp16

cfg_lora = SFTConfig(

    # <YOUR CODE HERE>

)

trainer_lora = SFTTrainer(
    model=lora_model,
    args=cfg_lora,
    train_dataset=train_ds_txt,
    eval_dataset=test_ds_txt,
    tokenizer=tokenizer,
    dataset_text_field="text",
)

# your code here: запустите обучение ᕙ(⇀‸↼‶)ᕗ

pass

# Compare outputs on EVAL_PROMPTS (baseline vs lora_sft)
lora_rows = []

for p in EVAL_PROMPTS:

    # your code here ヽ(♡‿♡)ノ

    pass

cmp_df = pd.DataFrame(lora_rows)

In [ ]:
# your code here: покажите несколько строк таблицы

Выберите 3 промпта из `EVAL_PROMPTS`, где изменение заметнее всего, и коротко ответьте:

1. что стало лучше/хуже после LoRA-SFT?
2. есть ли побочные эффекты (например: ответы стали длиннее, более шаблонными, хуже следуют инструкции)?

`# your text here (ಠ.ಠ)`

## Задание 6. Сравнение и анализ ошибок (1 балла)

После prompting и двух режимов обучения (LoRA-SFT и full SFT) важно сравнить модели одним и тем же числом, которое не зависит от вкуса проверяющего и не требует оценки глазами.

**Eval loss и perplexity на отложенной выборке** - это простой способ измерить, насколько модель лучше угадывает правильные ответы (в том формате, в котором мы ее обучали).

> Важно: loss не гарантирует, что ответы красивее или полезнее, но это честная метрика качества под ваш SFT-объектив.

**Этапы:**

1. Реализуйте функцию `eval_loss(model, ds, batch_size=4)`:

   * проходит по датасету батчами,
   * токенизирует поле `text`,
   * считает `loss` модели,
   * возвращает среднее значение loss.

2. Посчитайте `base_loss`, `lora_loss`, `full_loss` на `test_ds_txt`.

3. Посчитайте и выведите perplexit для каждой модели: `ppl = exp(loss)`.

In [ ]:
from transformers import DataCollatorForLanguageModeling

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

def eval_loss(model, ds, batch_size=4):
    model.eval()
    losses = []

    # your code here (•̀ ω •́ )✧

    return float(np.mean(losses))

# your code here: покажите результаты

Ответьте коротко:

- Почему важно считать loss на test, а не на train?
- Почему perplexity - это просто `exp(loss)` (в 1 предложении)?

`# your text here (ಠ.ಠ)`

### 6.2 Анализ ошибок

Ответьте одним-двумя абзацами:

1. На каких типах промптов LoRA дала улучшение сильнее, чем full fine-tuning (если такое есть)?
2. Где full fine-tuning оказался лучше?
3. Видите ли вы признаки переобучения (например: ответы стали слишком шаблонными/слишком длинными/ухудшилась общая адекватность)?
4. Какой метод вы бы выбрали в реальном проекте при ограниченном бюджете GPU - и почему?

**Ответ:**
`# your text here (ಠ.ಠ)`

## Задание 7. DPO (1 балл)

После SFT (supervised fine-tuning) модель учится копировать "правильные ответы" из датасета (instruction -> answer). Но в реальной жизни часто есть другая постановка: *нам важнее не единственно верный ответ, а предпочитаемый* - например:

* ответ должен быть **более вежливым**,
* **короче / структурнее**,
* **без опасных советов**,
* **более полезным**.

Для этого используют обучение по предпочтениям на парных данных: для одного и того же `prompt` есть два ответа:

* `chosen` - предпочтительный,
* `rejected` - менее предпочтительный.

**DPO (Direct Preference Optimization)** - простой и популярный способ дообучить модель так, чтобы она чаще предпочитала `chosen` и реже - `rejected`, *не вводя отдельную reward-модель и сложный RL-процесс*.

Интуитивно: мы подкручиваем вероятности токенов так, чтобы вероятность `chosen` росла относительно `rejected`.

<img src="https://miro.medium.com/v2/resize:fit:1400/1*lK6iJMz5CGh2fo7TsDn15A.png" alt="drawing" width="700"/>




**Что откуда брать?**

В этом задании нужны 4 вещи:

1. Ваша SFT-модель (желательно `lora_model` после LoRA-SFT из Задания 4) -> это будет `dpo_model`.

2. Reference model (ref_model) - якорь, замороженная копия модели до DPO. -> проще всего: загрузить заново `AutoModelForCausalLM.from_pretrained(MODEL_NAME, ...)`.

   Почему не брать `base_model` из памяти? Можно, но часто он уже жил в ноутбуке, и проще гарантировать чистоту новой загрузкой.

3. Preference-датасет с полями `prompt/chosen/rejected`. -> возьмём `trl-lib/ultrafeedback_binarized` (Hugging Face). Он большой, поэтому берем небольшой срез, например `train[:8000]`.

4. Тренер из TRL: `DPOTrainer` и конфиг `DPOConfig`. -> библиотека `trl` уже ставится у нас в Setting.


**Задание:**

1. Загрузите preference-датасет `trl-lib/ultrafeedback_binarized` и убедитесь, что у него есть (или сделайте через `map`) колонки:

   - `prompt`
   - `chosen`
   - `rejected`

2. Соберите `ref_model` (замороженная копия исходной модели) и `dpo_model` (ваша SFT-модель).

3. Соберите `DPOConfig` самостоятельно (как вы делали в SFT): подберите разумные значения `learning_rate`, `max_steps`, `gradient_accumulation_steps`, `beta`.

4. Запустите DPO-обучение.

5. Покажите 10–15 примеров до/после:

   - до - ответы вашей SFT-модели до DPO (сохраните копию или прогоните заранее),
   - после - ответы `dpo_model` после `trainer_dpo.train()`.

6. Коротко напишите вывод:

   - где стало лучше (в чём именно проявилось предпочтение),
   - где сломалось или стало хуже (типичные failure cases DPO на вашем датасете).

> Важно
>
> * DPO обычно запускают **поверх SFT**, иначе модель может дергаться сильнее.
>
> * Слишком большой `beta` делает обучение агрессивным (модель может стать однообразной).
>
> * Слишком длинный датасет/шаги = долго. Для ДЗ достаточно небольшого числа шагов.


In [ ]:
# Load preference dataset
from trl import DPOTrainer, DPOConfig
from datasets import load_dataset

pref_ds = load_dataset("trl-lib/ultrafeedback_binarized", split="train[:8000]")

# your code here (╯°□°）╯︵ ┻━┻
# Логика:
# 1) вывести column_names
# 2) если названия отличаются, сделать pref_ds = pref_ds.map(...)
# 3) колонки должны быть строго prompt / chosen / rejected

print(pref_ds)
print(pref_ds[0])

In [ ]:
# Build ref_model and dpo_model

# ref_model - замороженная копия модели ДО DPO
# dpo_model - ваша SFT-модель (LoRA-SFT)

# your code here: (⊙_⊙)
# Соберите ref_model и dpo_model
# убедитесь еще, что ref_model НЕ обучается

In [ ]:
# Configure DPO

# your code here: соберите конфиг сами (ง •̀_•́)ง

# Подсказка:
# - max_steps: 100–300 (в зависимости от времени)
# - lr: порядка 1e-6 .. 1e-5
# - beta: 0.05 .. 0.2
# - batch_size обычно 1

cfg_dpo = DPOConfig(
    output_dir="out_dpo",
    per_device_train_batch_size=1,
    gradient_accumulation_steps= # <YOUR CODE HERE>
    learning_rate= # <YOUR CODE HERE>
    max_steps= # <YOUR CODE HERE>
    beta= # <YOUR CODE HERE>
    logging_steps=25,
    save_strategy="no",
    fp16=(DTYPE == torch.float16)
)

In [ ]:
# Before/after set

# your code here: выберите 10–15 промптов из pref_ds
# (ﾉ◕ヮ◕)ﾉ*:･ﾟ✧

DPO_EVAL_PROMPTS = [
    # <YOUR CODE HERE>
]

# your code here: получите ответы "до DPO"
# Подсказка: используйте generate_text(dpo_model, prompt, system_text=SYSTEM_INSTR)
# и сохраните результаты в список
before_rows = []

In [ ]:
# Train DPO

trainer_dpo = DPOTrainer ... # your code here: ᕙ(⇀‸↼‶)ᕗ

In [ ]:
# After DPO + comparison table

# your code here: получите ответы "после DPO" на тех же DPO_EVAL_PROMPTS
after_rows = []

# your code here: соберите табличку сравнения
# columns: prompt / before / after
# и покажите ее

Вместо итога по DPO:
Напишите несколько предложений:
- где стало лучше и почему
- где стало хуже/ломается
- что бы поменяли в конфиге (beta/steps/lr), чтобы улучшить

`# your text here (ಠ.ಠ)`

Сделайте DPO поверх вашей SFT-модели (**рекомендуется начинать с LoRA-SFT**).

Нужно:

1. Загрузить preference-датасет (prompt/chosen/rejected) ([Hugging Face][2])
2. Запустить DPO-обучение
3. Показать **10–15 примеров “до/после”** и короткий вывод: стало ли лучше и где ломается

In [ ]:
from trl import DPOTrainer, DPOConfig

# Пример preference-датасета (можно заменить на другой)
pref_ds = load_dataset("trl-lib/ultrafeedback_binarized", split="train[:8000]")

# your code here: проверьте колонки pref_ds и при необходимости сделайте map
# нужно получить колонки: prompt / chosen / rejected
# (╯°□°）╯︵ ┻━┻

pref_ds

In [ ]:
# ref_model — "референс", обычно замороженная копия модели ДО DPO
ref_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=DTYPE
)
ref_model.eval()

# dpo_model — ваша SFT-модель (обычно LoRA-SFT удобнее)
dpo_model = lora_model
dpo_model.train()
dpo_model.config.use_cache = False

cfg_dpo = DPOConfig(
    output_dir="out_dpo",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=5e-6,
    max_steps=200,
    beta=0.1,
    logging_steps=25,
    save_strategy="no",
    fp16=(DTYPE == torch.float16)
)

trainer_dpo = DPOTrainer(
    model=dpo_model,
    ref_model=ref_model,
    args=cfg_dpo,
    train_dataset=pref_ds,
    tokenizer=tokenizer,
)

# your code here: trainer_dpo.train() ᕙ(⇀‸↼‶)ᕗ

In [ ]:
# your code here:

# 1) выберите 10–15 промптов (можно из pref_ds)
# 2) покажите "до DPO" (используйте LoRA-SFT модель до обучения или baseline)
# 3) покажите "после DPO" (dpo_model)
# 4) сохраните таблицу dpo_before_after.csv

# (ﾉ◕ヮ◕)ﾉ*:･ﾟ✧

## Заключение

Вы попробовали 4 подхода к адаптации одной LLM:

- Prompting baseline (без обучения)
- LoRA-SFT (parameter-efficient)
- Full fine-tuning (все веса)
- DPO (предпочтения)

**Вопрос:** Какой метод показал лучшее соотношение **качество / сложность / время / требования к ресурсам**?
Если бы вы делали продакшн-систему для автоматических ответов ассистента в компании (ограниченный GPU-бюджет), что бы вы выбрали и почему?

`your text here ( •̀ ω •́ )✧`

а ещё здесь ваши общие впечатления о домашней работе 🙏🏻